In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
from datetime import datetime, timedelta
import csv
import urllib.robotparser
from selenium import webdriver
import time

In [2]:
def phrase_filter(response: requests.models.Response):
    if '"incomplete_results":true' in response.text:
        return False
    return True 
requests_cache.install_cache(cache_name='demo_cache', filter_fn=phrase_filter)

In [5]:
driver = webdriver.Chrome()
driver2 = webdriver.Chrome()
url = ['https://soundcloud.com/search/sets?q=a', 'https://soundcloud.com/search/sets?q=b', 'https://soundcloud.com/search/sets?q=c', 'https://soundcloud.com/search/sets?q=d', 'https://soundcloud.com/search/sets?q=e', 'https://soundcloud.com/search/sets?q=g', 'https://soundcloud.com/search/sets?q=h', 'https://soundcloud.com/search/sets?q=i', 'https://soundcloud.com/search/sets?q=k']

# Write headers for file
with open('track.csv', 'w', encoding = "utf-8", newline = "\n") as outfile:
    writer = csv.writer(outfile, delimiter = "\t")  
    headers = ["Track", "Upload", "Play"]
    writer.writerow(headers)
with open('user.csv', 'w', encoding = "utf-8", newline = "\n") as outfile:
    writer = csv.writer(outfile, delimiter = "\t")  
    headers = ["Name", "Link", "Followers"]
    writer.writerow(headers)
with open('playlist.csv', 'w', encoding = "utf-8", newline = "\n") as outfile:
    writer = csv.writer(outfile, delimiter = "\t")  
    headers = ["Playlist", "Link", "Create", "Repost", "Like"]
    writer.writerow(headers)

for n in range(0,9):
    driver.get(url[n])
    time.sleep(5)
    i = 0
    while i < 20: 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        i += 1
        time.sleep(5)

    els = driver.find_elements_by_css_selector(".searchList__item")
    for x in range (0,len(els)):
        # Get user name: username and user link: href
        username = els[x].find_elements_by_css_selector(".soundTitle__usernameText")[0].text
        href = els[x].find_elements_by_css_selector(".soundTitle__username")[0].get_attribute("href")

        # Get playlist name: plname and playlists link: plhref 
        plhref = els[x].find_elements_by_css_selector(".soundTitle__title")[0].get_attribute("href")
        plname = els[x].find_elements_by_css_selector(".soundTitle__title")[0].find_elements_by_css_selector("span")[0].text

        driver2.get(plhref)
        time.sleep(2)

        plels = driver2.find_elements_by_css_selector("#content")
        for plitem in range(0, len(plels)):
            # Get time create playlist: timecreate
            timecreate = plels[plitem].find_elements_by_css_selector(".relativeTime")[0].get_attribute("title")

            # Get follower of user: followers
            UBadge = plels[plitem].find_elements_by_css_selector(".userBadge__content")[0]
            if UBadge.find_elements_by_css_selector(".sc-visuallyhidden"):
                followers = UBadge.find_elements_by_css_selector(".sc-visuallyhidden")[0].text.replace("followers"," ")
            else:
                followers = "None"

            # Get playlist like: plistlike    
            rates = plels[plitem].find_elements_by_css_selector(".listenEngagement__footer")[0]
            if rates.find_elements_by_css_selector(".sc-ministats-item"):
                plistlike = rates.find_elements_by_css_selector(".sc-ministats-item")[0].get_attribute("title").replace("likes"," ")
            else:
                plistlike = "None"

            # Get playlist repost: repost
            if rates.find_elements_by_css_selector(".sc-ministats-reposts"):
                repost = rates.find_elements_by_css_selector(".sc-ministats-reposts")[0].find_elements_by_css_selector("span")[1].text
            else:
                plistrepose = "None"

            # Write playlist.csv
            data2 = [plname, plhref, timecreate, repost, plistlike]
            with open("playlist.csv", 'a', encoding="utf-8", newline='\n') as f_op2:
                tsv_op2 = csv.writer(f_op2, delimiter='\t')
                tsv_op2.writerow(data2)
            
            # Write user.csv
            data3 = [username, href, followers]
            with open("user.csv", 'a', encoding="utf-8", newline='\n') as f_op3:
                tsv_op3 = csv.writer(f_op3, delimiter='\t')
                tsv_op3.writerow(data3)

            # Get track list in playlist
        tracklist = driver2.find_elements_by_css_selector(".trackList__item")
        for tr in range (1, len(tracklist)):
            # Get Track name: tracklistitem
            if tracklist[tr].find_elements_by_css_selector(".trackItem__trackTitle"):
                tracklistitem = tracklist[tr].find_elements_by_css_selector(".trackItem__trackTitle")[0].text
            else:
                break

            # Get Who is upload: upload
            if tracklist[tr].find_elements_by_css_selector(".trackItem__username"):
                    upload = tracklist[tr].find_elements_by_css_selector(".trackItem__username")[0].text
            else:
                upload = "None"

            # Get The number of play: playcount    
            if tracklist[tr].find_elements_by_css_selector(".trackItem__playCount"):
                playcount = tracklist[tr].find_elements_by_css_selector(".trackItem__playCount")[0].text
            else:
                playcount = "None"
              
            # Write tracks.csv
            data1=[tracklistitem, upload, playcount]
            with open("track.csv", 'a', encoding="utf-8", newline='\n') as f_op1:
                tsv_op = csv.writer(f_op1, delimiter='\t')
                tsv_op.writerow(data1)    

